
На прошлом шаге W’ имеет много нулей. Это снижает эффективность метода.

На этом шаге будет реализация через матрицы другим, более эффективным способом.

Пусть в этот раз на входе батч из одного трехслойного (RGB) изображения размером 3*3.

Пусть ядро имеет 2 фильтра шириной и высотой 2 пикселя.

Тогда выход должен иметь размерность 1*2*2*2.

Пусть W - веса ядра, X - значения входной матрицы, Y - значения на выходе.

Для простоты слои изображения и слои фильтров ядра покрашены в цвета.

**Обратите внимание**, например, "синяя" X0 не обязано быть равно "красному" X0, аналогично и про значения в фильтрах ядра - разный цвет и одинаковые переменные могут иметь разные значения, такое обозначение выбрано, чтобы не загромождать рисунок сложными индексами.

<img src='../data/5271.png'>

 

Если в первом матричном способе мы вытягивали изображения в столбцы, то теперь будем вытягивать фильтры кернела в строки.

 
<img src='../data/5272.svg'>


Рекомендуем проверить на листочке, что результат произведения таких матриц дает тот же результат, что и свертка.

Давайте перейдем от простого случая к общему:

- Если изображений в батче больше одного: преобразования ядра от этого не меняется, а преобразованные матрицы входных изображений конкатенируются по горизонтали.

 

Но рассчитанный таким способом выход не совпадает по размерности с выходом стандартного слоя из PyTorch - нужно изменить размерность.

 

Функция умножения матриц уже реализована.

 

Напоминание: во всех шагах этого урока мы считаем bias в сверточных слоях нулевым.

Требуется написать функции преобразования ядра и входа.

In [1]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        # превратим разные фильтры в строчки, входные каналы растянуты по столбцам
        converted_kernel = self.kernel.reshape([self.out_channels, self.in_channels * self.kernel_size ** 2])
        return converted_kernel

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        # пустая матрица куда будем вписывать куски входных данных
        converted_input = torch.zeros([self.kernel_size ** 2
                                       * self.in_channels,  # высота цветных кусков * количество цветов
                                      output_height * output_width 
                                       * torch_input.shape[0]])  # ширина цветных кусков * количество изображений
        # для всех изображений
        for image_ind in range(len(torch_input)):
            # идем по столбцам матрицы для одного изображения
            # как если бы они были элементами output
            for i in range(output_width):
                for j in range(output_height):
                    # берем срез который надо умножить на ядро
                    tmp = torch_input[image_ind,
                                       :,
                                       (i*self.stride):(i*self.stride+self.kernel_size),
                                       (j*self.stride):(j*self.stride+self.kernel_size)]
                    # вытягиваем его в столбец
                    converted_input[:,
                                    image_ind 
                                    * output_width
                                    * output_height
                                    + output_height*i 
                                    + j] = tmp.flatten()

        return converted_input

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.transpose(0, 1).view(torch_input.shape[0],
                                                     self.out_channels, output_height,
                                                     output_width).transpose(1, 3).transpose(2, 3)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dMatrixV2))